In [ ]:
#!pip install --user --upgrade pip

#!pip install opencv-python
#!pip install h5py
#!pip install keras

In [1]:
import cv2
import numpy as np
import h5py

import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import sys 
import os

sys.path.append(os.path.abspath("../"))

In [3]:
import lib

file_name = '../Etiquetado/bounds2.txt'
boundsdict, _ = lib.Utils.openBounds(file_name)

plot = False

input_scale = 0.6
output_scale = input_scale/8

cap = cv2.VideoCapture('../DCIM/saturated-noaudio.avi')

racap = lib.RACapWrapper(cap, 30)

dinput = []
doutput = []

for frame_n in sorted(boundsdict.keys()):    
    for bound in boundsdict[frame_n]:
        frame, _ = racap.readAt(frame_n)
        [height, width, _] = frame.shape      
        
        newinput = cv2.resize(frame, None, fx=input_scale, fy=input_scale, interpolation=cv2.INTER_NEAREST)
        newinput = newinput.astype('float32') / 255
               
        
        output = np.zeros((height, width, 1))
        cv2.rectangle(output, tuple(bound[0:2]), tuple(bound[2:4]), 1, cv2.FILLED)
        
        #Debe tener el mismo width que new input
        newoutput = cv2.resize(output, None, fx=input_scale, fy=output_scale, interpolation=cv2.INTER_NEAREST)
        newoutput = np.expand_dims(newoutput, axis=2).astype('float32')
        
        for x in range(int(width*input_scale)):
            dinput.append(newinput[:, x, :])        
            doutput.append(newoutput[:, x, :])                

cap.release()

data_fila = h5py.File('data1D.h5', 'w')

data_fila.create_dataset('dinput', data = dinput)
data_fila.create_dataset('doutput', data = doutput)

data_fila.close()